# Creating DCNN training libraries from label images

> Written by Dr Daniel Buscombe, Northern Arizona University

> Part of a series of notebooks for image recognition and classification using deep convolutional neural networks

This notebook is to demonstrate how to create libraries of categorized image tiles from ground truth (labelled) images, in order to retrain a DCNN using transfer learning

### Overview

The image is divided up into tiles of a specified size, "tilesize". 

If the proportion of pixels within the tile is greater than a specified amount, "threshold", then the tile is written to a file in a folder denoting its class. 

The tiles can then be 'thinned', by specifying what proportion of tiles (randomly selected from the entire catalogue) to keep - "prop_keep"

This generates both ground-truth label imagery (to evaluate classification performance) and sets of data suitable for training a DCNN.

![](figs/dl_tools_tile.png)

### Using a local directory

The images are from the California Coastal Records Project

Copyright (C) 2002–2018 Kenneth and Gabrielle Adelman, www.Californiacoastline.org and are used with permission. 

First, let's take a look at how many test/train images we have

In [ ]:
from glob import glob
files = glob('ccr_train/*.JPG')
len(files)

30 training images. How many test images?

In [ ]:
files = glob('ccr_test/*.JPG')
len(files)

we have 19 training images

Ok, let's create some 224x224 pixel tiles from the training images. We'll only keep tiles with 90% or more of pixels representing the dominant class, then we'll randomly select 10% of those tiles to train with (to speed things up)

In [ ]:
direc='ccr_train'
tilesize = 224
threshold = 0.9
prop_keep=0.125

Retiling takes a few minutes, so let's watch a video on one convolutional net layer

In [ ]:
from IPython.display import YouTubeVideo
## CNN example
YouTubeVideo('bXJx7y51cl0') ##11 mins

In [ ]:
## approx 6 mins

In [ ]:
%run ./retile.py $direc $tilesize $threshold $prop_keep

To do the same on the 'test' directory:

In [ ]:
direc='ccr_test'
tilesize = 224
threshold = 0.9
prop_keep=0.125

In [ ]:
## approx 4 mins

In [ ]:
%run ./retile.py $direc $tilesize $threshold $prop_keep 

### From S3 bucket

This is an example of how to do the same as the above but this time using a dataset on the S3 bucket

In [ ]:
import s3fs
fs = s3fs.S3FileSystem(anon=True)
S3direc='esipfed/cdi-workshop/semseg_data/ontario/train'
print(S3direc)

How many files?

In [ ]:
files = [f for f in fs.ls(S3direc) if f.endswith('.JPG')]
len(files)

In [ ]:
tilesize = 224
threshold = 0.9
prop_keep=0.25

The s3 bucket is a read-only filesystem, which means we have to write out the tiles here to your local directory

In [ ]:
from IPython.display import YouTubeVideo
## Why convolutions?
YouTubeVideo('ay3zYUeuyhU') ##9 mins

In [ ]:
%run ./retile_fromS3.py $S3direc $tilesize $threshold $prop_keep

Now we are ready to retrain the DCNN

## DL-tools

The equivalent function in DL-tools is called using:

```python create_library\retile.py -t 96 -a 0.9 -b 0.5```

where ```t``` is the tile size, ```a``` is the threshold proportion of pixels in a tile uniformly classified, and ```b``` is the proportion of the tiles to randomly select and keep

You will be prompted to select a set of .mat files. The tiles will be created in folders called "test" and "train"